In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np
import sys
import import_ipynb
import quickplay


Using TensorFlow backend.


In [3]:
x=quickplay.Simulation()
print("1 "+str(x.getBuyState(x.p1)))
print("2 "+str(x.getBuyState(x.p2)))
wins = [0]*100
for i in range(len(wins)):
    j=0
    while True:
        j+=1
        print(str(j)+" p1: "+str(x.makeDecision(x.p1,1)))
        if x.shop.checkEnd():
            print("shop end")
            print(x.shop.cards)
            break
        print(str(j)+" p2: "+str(x.makeDecision(x.p2,1)))
        if x.shop.checkEnd():
            print("shop end")
            print(x.shop.cards)
            break
    print(x.p1.getVP())
    print(x.p2.getVP())
    if x.p1.getVP() > x.p2.getVP():
        print(str(x.p1)+" wins!")
        wins[i]=1
    else:
        print(str(x.p2)+" wins!")
        wins[i]=2
    x=quickplay.Simulation()
print(str(wins))

Game Started
Copper purchased by Player 1
Estate purchased by Player 1
Copper purchased by Player 2
Estate purchased by Player 2
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 0, 0]
Player 2 wins!
1 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 1, 2]
2 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 1, 2]
purchase options: ['Cellar', 'Moat', 'Estate']
2
1
{'Cellar': 10, 'Market': 10, 'Militia': 10, 'Mine': 10, 'Moat': 10, 'Remodel': 10, 'Smithy': 10, 'Village': 10, 'Woodcutter': 10, 'Workshop': 10, 'Copper': 46, 'Silver': 40, 'Gold': 30, 'Estate': 8, 'Dutchy': 8, 'Province': 8}
False
Moat purchased by Player 1
1 p1: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 1, 5]
purchase options: ['Cellar', 'Moat', 'Estate']
2
1
{'Cellar': 10, 'Market': 10, 'Militia': 10, 'Mine': 10, 'Moat': 9, 'Remodel': 10, 'Smithy': 10, 'Village': 10, 'Woodcutter': 10, 'Workshop': 10, 'Copper': 46, 'Silver': 40, 'Gold': 30, 'Estate': 8, 'Dutchy': 8, 'Province': 8}
False
Cellar purchased by Player 2
1 p

False
Workshop purchased by Player 2
random action choice of Village
Village played by Player 2
random action choice of Market
Market played by Player 2
random action choice of Militia
Militia played by Player 2
random action choice of Remodel
Remodel played by Player 2
Gold purchased by Player 2
47 p2: [2, 3, 2, 4, 4, 4, 1, 1, 1, 2, 11, 4, 11, 6, 1, 4, 2, 5]
purchase options: ['Moat', 'Copper']
2
2
{'Cellar': 0, 'Market': 6, 'Militia': 5, 'Mine': 6, 'Moat': 1, 'Remodel': 1, 'Smithy': 7, 'Village': 5, 'Woodcutter': 5, 'Workshop': 1, 'Copper': 27, 'Silver': 30, 'Gold': 23, 'Estate': 0, 'Dutchy': 6, 'Province': 4}
False
Copper purchased by Player 1
48 p1: [8, 1, 3, 0, 5, 5, 2, 4, 4, 7, 19, 6, 0, 8, 1, 0, 1, 2]
purchase options: ['Market', 'Militia', 'Mine', 'Moat', 'Remodel', 'Smithy', 'Village', 'Woodcutter', 'Workshop', 'Copper', 'Silver', 'Dutchy']
5
2
{'Cellar': 0, 'Market': 6, 'Militia': 5, 'Mine': 6, 'Moat': 1, 'Remodel': 1, 'Smithy': 7, 'Village': 5, 'Woodcutter': 5, 'Workshop': 1

purchase options: ['Cellar', 'Moat', 'Estate']
2
1
{'Cellar': 9, 'Market': 10, 'Militia': 6, 'Mine': 9, 'Moat': 8, 'Remodel': 8, 'Smithy': 8, 'Village': 5, 'Woodcutter': 8, 'Workshop': 7, 'Copper': 38, 'Silver': 39, 'Gold': 30, 'Estate': 5, 'Dutchy': 7, 'Province': 8}
False
Estate purchased by Player 1
random action choice of Workshop
Workshop played by Player 1
Militia purchased by Player 1
16 p1: [1, 0, 2, 0, 0, 2, 1, 4, 1, 2, 12, 1, 0, 5, 1, 0, 1, 2]
purchase options: ['Cellar', 'Moat', 'Village', 'Woodcutter', 'Workshop', 'Silver', 'Estate']
3
1
{'Cellar': 9, 'Market': 10, 'Militia': 5, 'Mine': 9, 'Moat': 8, 'Remodel': 8, 'Smithy': 8, 'Village': 5, 'Woodcutter': 8, 'Workshop': 7, 'Copper': 38, 'Silver': 39, 'Gold': 30, 'Estate': 4, 'Dutchy': 7, 'Province': 8}
False
Estate purchased by Player 2
random action choice of Militia
Militia played by Player 2
16 p2: [0, 0, 3, 1, 2, 0, 1, 1, 1, 1, 9, 1, 0, 6, 0, 0, 1, 3]
purchase options: ['Cellar', 'Moat', 'Estate']
2
1
{'Cellar': 9, 'Mark

purchase options: ['Cellar', 'Moat', 'Estate']
2
1
{'Cellar': 7, 'Market': 8, 'Militia': 7, 'Mine': 9, 'Moat': 8, 'Remodel': 7, 'Smithy': 7, 'Village': 10, 'Woodcutter': 10, 'Workshop': 9, 'Copper': 40, 'Silver': 36, 'Gold': 30, 'Estate': 5, 'Dutchy': 7, 'Province': 8}
False
Moat purchased by Player 2
16 p2: [2, 1, 1, 1, 2, 2, 0, 0, 0, 1, 9, 2, 0, 6, 0, 0, 1, 4]
purchase options: ['Market', 'Militia', 'Mine', 'Remodel', 'Smithy', 'Dutchy']
5
1
{'Cellar': 7, 'Market': 8, 'Militia': 7, 'Mine': 9, 'Moat': 7, 'Remodel': 7, 'Smithy': 7, 'Village': 10, 'Woodcutter': 10, 'Workshop': 9, 'Copper': 40, 'Silver': 36, 'Gold': 30, 'Estate': 5, 'Dutchy': 7, 'Province': 8}
False
Smithy purchased by Player 1
17 p1: [1, 1, 2, 0, 1, 1, 4, 0, 0, 0, 11, 2, 0, 3, 1, 0, 1, 6]
purchase options: ['Militia', 'Remodel', 'Smithy', 'Village', 'Woodcutter', 'Workshop', 'Silver']
4
1
{'Cellar': 7, 'Market': 8, 'Militia': 7, 'Mine': 9, 'Moat': 7, 'Remodel': 7, 'Smithy': 6, 'Village': 10, 'Woodcutter': 10, 'Workshop'

False
Mine purchased by Player 2
random action choice of Cellar
Cellar played by Player 2
random action choice of Moat
Moat played by Player 2
16 p2: [5, 0, 1, 2, 2, 0, 3, 1, 1, 0, 7, 1, 0, 3, 0, 0, 1, 1]
purchase options: ['Cellar', 'Moat', 'Estate']
2
1
{'Cellar': 3, 'Market': 9, 'Militia': 9, 'Mine': 8, 'Moat': 3, 'Remodel': 9, 'Smithy': 7, 'Village': 8, 'Woodcutter': 8, 'Workshop': 8, 'Copper': 43, 'Silver': 39, 'Gold': 30, 'Estate': 6, 'Dutchy': 7, 'Province': 8}
False
Estate purchased by Player 1
random action choice of Moat
Moat played by Player 1
17 p1: [2, 1, 0, 0, 5, 1, 0, 1, 1, 2, 9, 1, 0, 6, 1, 0, 1, 2]
purchase options: ['Copper']
1
1
{'Cellar': 3, 'Market': 9, 'Militia': 9, 'Mine': 8, 'Moat': 3, 'Remodel': 9, 'Smithy': 7, 'Village': 8, 'Woodcutter': 8, 'Workshop': 8, 'Copper': 43, 'Silver': 39, 'Gold': 30, 'Estate': 5, 'Dutchy': 7, 'Province': 8}
False
Copper purchased by Player 2
random action choice of Smithy
Smithy played by Player 2
17 p2: [5, 0, 1, 2, 2, 0, 3, 1, 1, 

...

24 p2: [4, 1, 0, 1, 5, 1, 1, 2, 4, 2, 7, 3, 2, 7, 2, 0, 1, 2]
purchase options: ['Militia', 'Remodel', 'Smithy', 'Village', 'Woodcutter', 'Workshop', 'Silver']
4
1
{'Cellar': 4, 'Market': 8, 'Militia': 8, 'Mine': 5, 'Moat': 0, 'Remodel': 8, 'Smithy': 9, 'Village': 5, 'Woodcutter': 6, 'Workshop': 7, 'Copper': 39, 'Silver': 36, 'Gold': 29, 'Estate': 1, 'Dutchy': 5, 'Province': 8}
False
Workshop purchased by Player 1
random action choice of Remodel
Remodel played by Player 1
Silver purchased by Player 1
25 p1: [2, 1, 2, 4, 5, 1, 0, 3, 0, 2, 9, 4, 2, 6, 1, 0, 1, 0]
purchase options: ['Cellar', 'Estate']
2
1
{'Cellar': 4, 'Market': 8, 'Militia': 8, 'Mine': 5, 'Moat': 0, 'Remodel': 8, 'Smithy': 9, 'Village': 5, 'Woodcutter': 6, 'Workshop': 6, 'Copper': 39, 'Silver': 35, 'Gold': 29, 'Estate': 1, 'Dutchy': 5, 'Province': 8}
False
Estate purchased by Player 2
random action choice of Cellar
Cellar played by Player 2
random action choice of Remodel
Remodel played by Player 2
25 p2: [4, 1, 0, 1, 5

25 p1: [1, 1, 3, 1, 0, 1, 1, 3, 2, 4, 14, 4, 0, 4, 1, 0, 1, 4]
purchase options: ['Militia', 'Remodel', 'Smithy', 'Village', 'Woodcutter', 'Workshop', 'Silver']
4
1
{'Cellar': 8, 'Market': 9, 'Militia': 7, 'Mine': 9, 'Moat': 6, 'Remodel': 8, 'Smithy': 9, 'Village': 4, 'Woodcutter': 5, 'Workshop': 6, 'Copper': 31, 'Silver': 38, 'Gold': 30, 'Estate': 3, 'Dutchy': 5, 'Province': 8}
False
Silver purchased by Player 2
25 p2: [1, 0, 0, 0, 4, 1, 0, 3, 3, 0, 13, 1, 0, 7, 2, 0, 1, 1]
purchase options: ['Militia', 'Remodel', 'Smithy', 'Village', 'Woodcutter', 'Workshop', 'Silver']
4
1
{'Cellar': 8, 'Market': 9, 'Militia': 7, 'Mine': 9, 'Moat': 6, 'Remodel': 8, 'Smithy': 9, 'Village': 4, 'Woodcutter': 5, 'Workshop': 6, 'Copper': 31, 'Silver': 37, 'Gold': 30, 'Estate': 3, 'Dutchy': 5, 'Province': 8}
False
Workshop purchased by Player 1
random action choice of Militia
Militia played by Player 1
26 p1: [1, 1, 3, 1, 0, 1, 1, 3, 2, 5, 14, 4, 0, 4, 1, 0, 1, 6]
purchase options: ['Copper']
1
1
{'Cellar'

random action choice of Woodcutter
Woodcutter played by Player 1
6 p1: [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 8, 0, 0, 4, 1, 0, 2, 4]
purchase options: ['Copper']
0
1
{'Cellar': 9, 'Market': 9, 'Militia': 10, 'Mine': 10, 'Moat': 10, 'Remodel': 8, 'Smithy': 10, 'Village': 9, 'Woodcutter': 9, 'Workshop': 8, 'Copper': 44, 'Silver': 40, 'Gold': 30, 'Estate': 6, 'Dutchy': 7, 'Province': 8}
False
Copper purchased by Player 2
6 p2: [1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 9, 0, 0, 4, 0, 0, 1, 3]
purchase options: ['Cellar', 'Militia', 'Moat', 'Remodel', 'Smithy', 'Village', 'Woodcutter', 'Workshop', 'Copper', 'Silver', 'Estate']
4
2
{'Cellar': 9, 'Market': 9, 'Militia': 10, 'Mine': 10, 'Moat': 10, 'Remodel': 8, 'Smithy': 10, 'Village': 9, 'Woodcutter': 9, 'Workshop': 8, 'Copper': 43, 'Silver': 40, 'Gold': 30, 'Estate': 6, 'Dutchy': 7, 'Province': 8}
False
Moat purchased by Player 1
7 p1: [0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 8, 0, 0, 4, 1, 0, 1, 2]
purchase options: ['Cellar', 'Moat', 'Village', 'Woodcutter', 'Worksh

{'Cellar': 10, 'Market': 10, 'Militia': 10, 'Mine': 10, 'Moat': 10, 'Remodel': 10, 'Smithy': 10, 'Village': 10, 'Woodcutter': 10, 'Workshop': 10, 'Copper': 46, 'Silver': 40, 'Gold': 30, 'Estate': 8, 'Dutchy': 8, 'Province': 8}
False
Estate purchased by Player 1
1 p1: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 4, 0, 0, 1, 5]
purchase options: ['Cellar', 'Moat', 'Estate']
2
1
{'Cellar': 10, 'Market': 10, 'Militia': 10, 'Mine': 10, 'Moat': 10, 'Remodel': 10, 'Smithy': 10, 'Village': 10, 'Woodcutter': 10, 'Workshop': 10, 'Copper': 46, 'Silver': 40, 'Gold': 30, 'Estate': 7, 'Dutchy': 8, 'Province': 8}
False
Cellar purchased by Player 2
1 p2: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 3, 0, 0, 1, 5]
purchase options: ['Market', 'Militia', 'Mine', 'Remodel', 'Smithy', 'Dutchy']
5
1
{'Cellar': 9, 'Market': 10, 'Militia': 10, 'Mine': 10, 'Moat': 10, 'Remodel': 10, 'Smithy': 10, 'Village': 10, 'Woodcutter': 10, 'Workshop': 10, 'Copper': 46, 'Silver': 40, 'Gold': 30, 'Estate': 7, 'Dutchy': 8, 'Province

ValueError: empty range for randrange() (0,0, 0)

In [53]:
#print(wins.count(1))
#print(wins.count(2))
print(x.shop.cards)
print(len(list(filter(lambda y: x.shop.cards[y]==0 , x.shop.cards)))>=3 or x.shop.cards["Province"]==0 )
print(x.shop.checkEnd())

{'Cellar': 0, 'Market': 4, 'Militia': 3, 'Mine': 10, 'Moat': 0, 'Remodel': 0, 'Smithy': 3, 'Village': 0, 'Woodcutter': 4, 'Workshop': 0, 'Copper': 17, 'Silver': 33, 'Gold': 26, 'Estate': 0, 'Dutchy': 7, 'Province': 6}
True
False


In [18]:
model = Sequential()
model.add(Dense(10, activation='sigmoid', input_shape=(18,)))
model.add(Dense(2, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [19]:
y = 0.95
eps = 0.45
decay_factor = 0.999
r_avg_list = []
num_episodes = 50
for i in range(num_episodes):
    s = env.reset()
    eps *= decay_factor
    if i % 5 == 0:
        print("Episode {} of {}".format(i + 1, num_episodes))
    done = False
    r_sum = 0
    while not done:
        if np.random.random() < eps:
            a = np.random.randint(0, 2)
        else:
            a = np.argmax(model.predict(np.identity(5)[s:s + 1]))
        new_s, r, done, _ = env.step(a)
        target = r + y * np.max(model.predict(np.identity(5)[new_s:new_s + 1]))
        target_vec = model.predict(np.identity(5)[s:s + 1])[0]
        target_vec[a] = target
        model.fit(np.identity(5)[s:s + 1], target_vec.reshape(-1, 2), epochs=1, verbose=0)
        s = new_s
        r_sum += r
    r_avg_list.append(r_sum / 1000)

NameError: name 'env' is not defined